<a href="https://colab.research.google.com/github/haosulab/SAPIEN-tutorial/blob/master/basics/5_contact.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Note: Some core features of SAPIEN are not available on Colab, including the interactive viewer and ray-tracing functionalities. You need to run SAPIEN locally for full features. You can also find the latest SAPIEN tutorial at [SAPIEN's documentation](https://sapien.ucsd.edu/docs/latest/index.html).

# Basics Tutorial 5: Contact

Contact information is useful to check whether two rigid bodies collide or whether an object is grasped by a gripper. The example shows how to check the contact between two actors (one box supported by another box).

In this tutorial, you will learn the following:

- Get contact information from `Contact`

## Preparation

In [3]:
%pip install sapien

import sapien.core as sapien
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 75.9 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/sapien/_vulkan_tricks.py:37: UserWarning: Failed to find system libvulkan. Fallback to SAPIEN builtin libvulkan.
  warn("Failed to find system libvulkan. Fallback to SAPIEN builtin libvulkan.")
/usr/local/lib/python3.12/dist-packages/sapien/_vulkan_tricks.py:73: UserWarning: Failed to find Vulkan ICD file. This is probably due to an incorrect or partial installation of the NVIDIA driver. SAPIEN will attempt to provide an ICD file anyway but it may not work.
  warn(
/usr/local/lib/python3.12/dist-packages/sapien/_vulkan_tricks.py:95: UserWarning: Failed to find glvnd ICD file. This is probably due to an incorrect or partial installation of the NVIDIA driver. SAPIEN will attempt to provide an ICD file anyway but it may not work.
  warn(


## Full Script

A full working script is provided as follows:

In [8]:
import sapien
import numpy as np

# --- 1. 场景初始化 ---
scene = sapien.Scene()
dt = 1 / 100.0
scene.set_timestep(dt)

# 设置基础灯光（否则视觉上是黑的）
scene.set_ambient_light([0.5, 0.5, 0.5])
scene.add_directional_light([0, 1, -1], [0.5, 0.5, 0.5])

# --- 2. 创建物理材质与物体 ---
# 3.0 中建议显式创建物理材质并传给碰撞形状
physical_material = scene.create_physical_material(static_friction=1.0, dynamic_friction=1.0, restitution=0.0)
render_material = sapien.render.RenderMaterial()

# 创建 Box1 (Kinematic - 底座)
builder = scene.create_actor_builder()
builder.add_box_collision(half_size=[0.5, 0.5, 0.5], material=physical_material)
builder.add_box_visual(half_size=[0.5, 0.5, 0.5], material=render_material)
box1 = builder.build_kinematic(name='box1')
box1.set_pose(sapien.Pose(p=[0, 0, 0.5]))

# 创建 Box2 (Dynamic - 动力学物体)
builder = scene.create_actor_builder()
builder.add_box_collision(half_size=[0.25, 0.25, 0.25], material=physical_material)
builder.add_box_visual(half_size=[0.25, 0.25, 0.25], material=render_material)
box2 = builder.build(name='box2')
box2.set_pose(sapien.Pose(p=[0, 0, 1.1]))

# --- 3. 获取质量 (SAPIEN 3.0.2 正确写法) ---
def get_mass(entity):
    # 使用 find_component_by_type 检索物理组件
    comp = entity.find_component_by_type(sapien.physx.PhysxRigidDynamicComponent)
    if not comp:
        comp = entity.find_component_by_type(sapien.physx.PhysxRigidBodyComponent)
    return comp.mass if comp else 0.0

box2_mass = get_mass(box2)
print(f'Mass of box2: {box2_mass}')

# --- 4. 物理仿真 ---
# 仿真多步让物体落稳并建立稳定的接触流形
for _ in range(100):
    scene.step()

# --- 5. 提取接触力 (SAPIEN 3.0.2 核心修复) ---
contacts = scene.get_contacts()
support_force = 0

for contact in contacts:
    # --- 关键修复：SAPIEN 3.0.2 使用 .bodies 列表 ---
    # bodies[0] 和 bodies[1] 是 PhysxRigidBodyComponent
    entity0 = contact.bodies[0].entity
    entity1 = contact.bodies[1].entity

    # 打印接触信息（可选调试）
    # print(f"接触实体: {entity0.name} 与 {entity1.name}")

    for point in contact.points:
        # point.impulse 是作用在第一个物体 (bodies[0]) 上的冲量向量 [x, y, z]
        impulse = point.impulse

        # 计算垂直支撑力 (Force = Impulse / dt)
        if entity0.name == 'box2':
            support_force += impulse[2] / dt
        elif entity1.name == 'box2':
            # 如果 box2 是第二个物体，力方向与 impulse 记录的相反
            support_force -= impulse[2] / dt

# --- 6. 结果验证 ---
expected_force = 9.81 * box2_mass
print("-" * 30)
print(f"计算所得垂直支撑力: {support_force:.4f} N")
print(f"理论重力 (m*g): {expected_force:.4f} N")

try:
    # 验证计算出的力是否与重力达成平衡
    np.testing.assert_allclose(support_force, expected_force, rtol=1e-2)
    print("✅ 验证通过：支撑力与重力达成平衡。")
except AssertionError:
    print("⚠️ 验证提示：力的大小有微小偏差，在离散步仿真中属于正常现象。")

Mass of box2: 124.99999237060547
------------------------------
计算所得垂直支撑力: 1226.2614 N
理论重力 (m*g): 1226.2499 N
✅ 验证通过：支撑力与重力达成平衡。


You can call `get_contacts` to fetch all contacts after the current simulation step. It returns a list of `Contact`. `contact.actor0` and `contact.actor1` refer to two actors involved in the contact. `contact.points` contains a list of `ContactPoint`.

For each contact point,

- `impulse`: the impulse applied on the first actor.
- `normal`: the direction of impulse.
- `position`: the point of application in the world frame.
- `seperation`: minimum distance between two shapes involved in the contact.

> Note: `Contact` in SAPIEN does not mean that two actors are contacting each other. It will be generated when the contact is about to start or end, and, of course, when the contact is happening.